<a href="https://colab.research.google.com/github/GustavSB/A5/blob/master/oliveti_faces_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets.olivetti_faces import fetch_olivetti_faces
from keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Activation, Dropout
from tensorflow.keras.utils import plot_model
from keras import regularizers
from keras.losses import mse, binary_crossentropy
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [ ]:
  
# Fetch data and have a look
faces = fetch_olivetti_faces()
x, y = faces['data'], faces['target']

# Dimensions
print(f'Data shape: {x.shape}')
print(f'Label shape: {y.shape}')
# (400, 4096)
# (400,)

print(np.unique(y))

# create a grid of 3x3 images
for i in range(0, 9):
    plt.subplot(330 + 1 + i)
    plt.imshow(x[i].reshape(64, 64))

# show the plot
plt.show()

# divide data into training, validation and testing (shuffle)
#
# Important notes:
# test_size=0.2 Holdout. Reserves 20% of the data for testing. 
# random_state=42 provides the shuffle(randomizer) function with a seed number. 
# stratify=y will distribute the data 'equally' meaning, it samples the whole dataset. The opposite would be to say, divide the data based
# it's meaningful representation, clustered images in this case. So of we trained on one part of the clustered imgs, and tested on the rest, 
# the model would be useless, as it would be overfitting. 
scaler = MinMaxScaler()
x = scaler.fit_transform(x)
xtrain, xtest, ytrain, ytest =train_test_split(x,y, stratify=y, test_size=0.2, random_state=42)

print(xtrain.shape) # 80% of the pixels in an image
print(xtest.shape) # 20% of the pixels in an image
print(y[0:10])
print('training data', ytrain[0:10])
print('testing data', ytest[0:10])

print(np.bincount(y)) # 10 pictures each person, 40 persons - 400 pictures. 
print(np.bincount(ytrain)) # The 8 pictures per person for training
print(np.bincount(ytest)) # The 2 pictures for testing


# New Section

In [ ]:
autoencoder = Sequential()
autoencoder.add(Dense(units = 1024, activation='relu', input_dim = 4096, name='encoder_layer1'))   # encoder layers
autoencoder.add(Dense(units = 512, activation='relu', input_dim =  1024, name='encoder_layer2'))
autoencoder.add(Dense(units =  64, activation='relu', input_dim =  512, name='encoder_layer3'))


autoencoder.add(Dense(units = 512, activation='relu', name='decoder_layer1'))                    # decoder layers
autoencoder.add(Dense(units = 1024, activation='relu', name='decoder_layer2'))
autoencoder.add(Dense(units = 4096, activation='sigmoid', name='decoder_layer3'))    #linear - softmax

autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])   #mean_squared_error

autoencoder.summary()
plot_model(autoencoder, to_file='AE_plot.png', show_shapes=True, show_layer_names=True)

from IPython.display import Image 
Image('AE_plot.png')

history = autoencoder.fit(x = xtrain, y= xtrain, epochs=1000, batch_size=32, shuffle=True,\
                    validation_data=(xtrain, xtrain), verbose=1)


In [ ]:
# test trained autoencoder
encoder = Model(inputs = autoencoder.input, outputs = autoencoder.layers[2].output)
encoded_imgs = encoder.predict(xtest)

print(encoded_imgs.shape)

# retrieve the last layer of the autoencoder model

encoded_input = Input(shape=(64,))
decoder_layer = autoencoder.layers[3](encoded_input)
decoder_layer = autoencoder.layers[4](decoder_layer)
decoder_layer = autoencoder.layers[5](decoder_layer)

decoder = Model(inputs = encoded_input, outputs = decoder_layer)

decoded_imgs = decoder.predict(encoded_imgs)

print(encoded_imgs.shape)

predicted_imgs = autoencoder.predict(xtest, verbose=1)

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(xtest[i].reshape(64, 64))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display encoded images
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(encoded_imgs[i].reshape(8, 8))
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(3, n, i + 1 + 2*n)
    plt.imshow(decoded_imgs[i].reshape(64, 64))
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:

# PCA DIMENSIONALITY REDUCTION AND REPRODUCTION
#
# PCA Principal Component Analysis will reduce the image dimensionally and pick out the most important data points/ pixel values
# meaning it will reduce the data amount yet keep the 'necessary tools' to reconstruct the images but with way less data. 
# The amount of loss in the reproduced images can be increased or reduced, based on whatever level of accuracy is satisfactory. 
# From the lectures we discussed the fact that with only three pixel values or something we would be able to reproduce at an accuracy of about 50%,
# which is..not terrible to be honest.
# I talk about 'accuracy', in more precise terms it is the totalt variance, meaning the squared distance from the datapoints to the component(s)
# where the least squared distance is preferable of course. 

pca = PCA(n_components=8*8) # Can also set a 'target accuracy' 0.8, number of components will be set automatically to achieve 80%.
pca.fit(x)
cumsum = np.cumsum(pca.explained_variance_ratio_)

plt.figure()
plt.plot(range(len(cumsum)), cumsum)
plt.grid()
plt.xlabel('Principal components - amount of necessary pixels / data points')
plt.ylabel('Explained variance - accuracy')
plt.show()
print('At 64x64 pixels (64 pca) we will get a 90% accuracy')

x_reduced = pca.fit_transform(xtest)
x_recovered = pca.inverse_transform(x_reduced)

#print(x_reduced)
#print(x_recovered)

def display_val(x):
  fig, ax = plt.subplots(3,5)
  ax = ax.flatten()
  ims = []
  for i in range(len(ax)):
    ax[i].imshow(xtest[i].reshape(64,64), interpolation='nearest', animated=True)
    ax[i].xaxis.set_visible(False)
    ax[i].yaxis.set_visible(False)

display_val(xtest)
display_val(x_recovered)


In [ ]:

# unsupervised algorithm
#kmeans is an unsupervised algorithm
kmeans = KMeans(n_clusters=40, max_iter=10000, algorithm = 'auto', random_state=7, verbose=1)

#train 
encoded_imgs = encoder.predict(xtrain)
kmeans.fit(encoded_imgs)

encoded_imgs = encoder.predict(xtest)
ypred = kmeans.predict(encoded_imgs)
print(ypred[0,])
print(ytest[0,])

# Model Accuracy, how often is the classifier correct?
print("Accuracy:", accuracy_score(ytest, ypred))

cmap = ListedColormap(['lightgrey', 'silver', 'ghostwhite', 'lavender', 'wheat'])
plt.rcParams["figure.figsize"] = (8,8)


In [ ]:

# supervised algorithm


ytrain_ = to_categorical(ytrain)
ytest_ = to_categorical(ytest)


model = Sequential()
model.add(Dense(56, activation='relu', input_shape=(8*8, ), name='hidden_layer'))
model.add(Dense(40, activation='softmax', name='output_layer'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()
plot_model(model, to_file='mnist_keras.png', show_shapes=True, show_layer_names=True)

enc_train = encoder.predict(xtrain).reshape(320,8*8)
enc_test =encoder.predict(xtest).reshape(80,8*8)
print(enc_train.shape)
print(enc_test.shape)


history = model.fit(enc_train, ytrain_, epochs=100, batch_size=32)
test_loss, test_accu = model.evaluate(enc_test, ytest_)
print(test_loss, test_accu)

cmap = ListedColormap(['lightgrey', 'silver', 'ghostwhite', 'lavender', 'wheat'])
plt.rcParams["figure.figsize"] = (8,8)

# confusion matrix
def cm(ytest, ypred, title):
  print(ytest.shape)
  print(ypred.shape)

  cm = confusion_matrix(ytest, ypred)

  plt.figure()
  plt.matshow(cm, cmap=cmap)
  
  for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
      plt.text(x=j, y=i, s=cm[i,j], va='center', ha='center')
  
  plt.title(title)
  plt.xlabel('Predicted label')
  plt.ylabel('True label')
  plt.show()


ypred_train = model.predict(enc_train)
ypred_train = np.argmax(ypred_train, axis=-1)
cm(ytrain, ypred_train, title='Train')



ypred_test = model.predict(enc_test)
ypred_test = np.argmax(ypred_test, axis=-1)
print(ypred_test.shape)
print(ytest.shape)

print(np.unique(ypred_test))
print(np.unique(ytest))

print('Precision score:', precision_score(ytest, ypred_test, average='macro'))
print('Recall score   :', recall_score(ytest, ypred_test, average='macro'))

cm(ytest[1:], ypred_test[1:], title='Test')


In [ ]:
def cm(ytest, ypred, title):
  cm = confusion_matrix(ytest, ypred)

  plt.figure()
  plt.matshow(cm, cmap=cmap)
  
  for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
      plt.text(x=j, y=i, s=cm[i,j], va='center', ha='center')
  
  plt.title(title)
  plt.xlabel('Predicted label')
  plt.ylabel('True label')
  plt.show()

cm(ytrain, kmeans.predict(encoder.predict(xtrain)), title='Train')
cm(ytest, kmeans.predict(encoder.predict(xtest)), title='Test')

In [ ]:
#cross-validation  - SGD, BGD and MBGD
